# <font color='CC3D3D'> 0.5_9 Merge_Path_Clustering
    
- 비슷한 경로의 6개의 군집으로 나눈 path_group.csv와 loan_result.csv를 취합한 코드
- 중간 단계의 데이터 <span style="color:blue"> **loan_.csv** </span> 생성
- 최종 취합 데이터 <span style="color:blue"> **loan_path_segment.csv** </span> 생성

# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
from IPython.display import display
import gc
from typing import Union
import time

pd.set_option('display.max_columns', None)

with open("../Data/dtypes.pkl", 'rb') as f:
    dtypes = pickle.load(f)

In [ ]:
def view_info(df:pd.DataFrame) -> None:
    print(df.shape)
    print()
    df.info()
    print()
    display(df.head())

# Task

## Data Import

In [ ]:
loan = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/loan_result.csv", engine='c',
                   low_memory=True,
                   parse_dates=['loanapply_insert_time'],
                   dtype={'application_id': 'uint32', 'bank_id': 'uint8', 'product_id': 'uint8'},
                   usecols=['application_id', 'loanapply_insert_time', 'is_applied'])
log = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/log_data.csv", engine='c', 
                  dtype={'user_id': 'uint32', 'event': 'object'},
                  usecols=['user_id', 'event', 'timestamp', 'date_cd'], 
                  low_memory=True,
                  parse_dates=['timestamp', 'date_cd'])
user = pd.read_csv("../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/user_spec.csv", engine='c',
                   dtype={'application_id': 'uint32', 'user_id': 'uint32'},
                   low_memory=True,
                   parse_dates=['insert_time'],
                   usecols=['application_id', 'user_id', 'insert_time'])

view_info(loan)
print()
view_info(log)
print()
view_info(user)

(13527363, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527363 entries, 0 to 13527362
Data columns (total 3 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
dtypes: datetime64[ns](1), float64(1), uint32(1)
memory usage: 258.0 MB



,application_id,loanapply_insert_time,is_applied
0,1748340,2022-06-07 13:05:41,NaN
1,1748340,2022-06-07 13:05:41,NaN
2,1748340,2022-06-07 13:05:41,NaN
3,1748340,2022-06-07 13:05:41,NaN
4,1748340,2022-06-07 13:05:41,NaN



(17843993, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17843993 entries, 0 to 17843992
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    uint32        
 1   event      object        
 2   timestamp  datetime64[ns]
 3   date_cd    datetime64[ns]
dtypes: datetime64[ns](2), object(1), uint32(1)
memory usage: 476.5+ MB



,user_id,event,timestamp,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,2022-03-25



(1394216, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394216 entries, 0 to 1394215
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   application_id  1394216 non-null  uint32        
 1   user_id         1394216 non-null  uint32        
 2   insert_time     1394216 non-null  datetime64[ns]
dtypes: datetime64[ns](1), uint32(2)
memory usage: 21.3 MB



,application_id,user_id,insert_time
0,1249046,118218,2022-06-07 06:28:18
1,954900,553686,2022-06-07 14:29:03
2,137274,59516,2022-06-07 21:40:22
3,1570936,167320,2022-06-07 09:40:27
4,967833,33400,2022-06-07 08:55:07


In [ ]:
loan.isnull().sum()

application_id                 0
loanapply_insert_time          0
is_applied               3257239
dtype: int64

## Merge application - user

In [ ]:
# app_user = {user['application_id'].iloc[i]: user['user_id'].iloc[i] for i in range(user.shape[0])}
# loan['user_id'] = loan['application_id'].map(app_user)
# loan.isnull().sum()
# train = loan[loan['loanapply_insert_time'].dt.month < 6]
# train['user_id'] = train['application_id'].map(app_user)
# train.dropna(subset=['user_id'], inplace=True)
# train['user_id'] = train['user_id'].astype(np.uint32)

# view_info(train)

application_id                 0
loanapply_insert_time          0
is_applied               3257239
user_id                      113
dtype: int64

In [ ]:
loan.dropna(subset=['user_id'], inplace=True)

In [ ]:
# train = loan[loan['loanapply_insert_time'].dt.month < 6]
# apply_group = train[['application_id', 'is_applied']].groupby(by='application_id').sum()
# apply_group.reset_index(drop=False, inplace=True)
# apply_group.columns = ['application_id', 'total_applied']

# view_info(apply_group)

(728997, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728997 entries, 0 to 728996
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   application_id  728997 non-null  uint64 
 1   total_applied   728997 non-null  float64
dtypes: float64(1), uint64(1)
memory usage: 11.1 MB



,application_id,total_applied
0,1,0.0
1,2,0.0
2,6,0.0
3,14,3.0
4,17,1.0


In [ ]:
total_loan = pd.merge(left=train, right=apply_group, on='application_id', how='inner')
view_info(total_loan)

(10270124, 4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10270124 entries, 0 to 10270123
Data columns (total 4 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   total_applied          float64       
dtypes: datetime64[ns](1), float64(2), uint32(1)
memory usage: 352.6 MB



,application_id,loanapply_insert_time,is_applied,total_applied
0,2157865,2022-05-09 08:44:59,1.0,1.0
1,576643,2022-05-09 10:54:53,0.0,0.0
2,576643,2022-05-09 10:54:53,0.0,0.0
3,2136706,2022-05-09 10:41:06,0.0,3.0
4,2136706,2022-05-09 10:41:07,0.0,3.0


In [ ]:
total_loan['user_id'] = total_loan['application_id'].map(app_user)
total_loan.dropna(subset=['user_id'], inplace=True)
total_loan['user_id'] = total_loan['user_id'].astype(np.uint32)
total_loan['date_cd'] = pd.to_datetime(total_loan['loanapply_insert_time'].dt.date)
view_info(total_loan)

(10270011, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10270011 entries, 0 to 10270123
Data columns (total 6 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   total_applied          float64       
 4   user_id                uint32        
 5   date_cd                datetime64[ns]
dtypes: datetime64[ns](2), float64(2), uint32(2)
memory usage: 470.1 MB



,application_id,loanapply_insert_time,is_applied,total_applied,user_id,date_cd
0,2157865,2022-05-09 08:44:59,1.0,1.0,346970,2022-05-09
1,576643,2022-05-09 10:54:53,0.0,0.0,545882,2022-05-09
2,576643,2022-05-09 10:54:53,0.0,0.0,545882,2022-05-09
3,2136706,2022-05-09 10:41:06,0.0,3.0,558819,2022-05-09
4,2136706,2022-05-09 10:41:07,0.0,3.0,558819,2022-05-09


In [ ]:
log_train = log[log['date_cd'].dt.month < 6]
view_info(log_train)

(12907328, 4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12907328 entries, 0 to 17843992
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    uint32        
 1   event      object        
 2   timestamp  datetime64[ns]
 3   date_cd    datetime64[ns]
dtypes: datetime64[ns](2), object(1), uint32(1)
memory usage: 443.1+ MB



,user_id,event,timestamp,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,2022-03-25


In [ ]:
app_log_merge = pd.merge(left=log_train, right=total_loan[['user_id', 'application_id', 'date_cd', 'total_applied']], on=['user_id', 'date_cd'], how='inner')
app_log_merge['total_applied'] = app_log_merge['total_applied'].astype(np.uint8)
view_info(app_log_merge)

(105414947, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105414947 entries, 0 to 105414946
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         uint32        
 1   event           object        
 2   timestamp       datetime64[ns]
 3   date_cd         datetime64[ns]
 4   application_id  uint32        
 5   total_applied   uint8         
dtypes: datetime64[ns](2), object(1), uint32(2), uint8(1)
memory usage: 4.0+ GB



,user_id,event,timestamp,date_cd,application_id,total_applied
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
1,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
2,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
3,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
4,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0


## Split into actual applied vs non-applied

In [ ]:
app_id = apply_group[apply_group['total_applied'] != 0]['application_id'].unique().astype(np.uint32)
noapp_id = apply_group[apply_group['total_applied'] == 0]['application_id'].unique().astype(np.uint32)

print(f"# of application ids that applied: {app_id.shape[0]}")
print(f"# of application ids that not applied: {noapp_id.shape[0]}")

print(app_id.max())
print(noapp_id.max())

# of application ids that applied: 377669
# of application ids that not applied: 351328
2167826
2167819


In [ ]:
actual_log = app_log_merge[app_log_merge['application_id'].isin(app_id)]
no_log = app_log_merge[app_log_merge['application_id'].isin(noapp_id)]

In [ ]:
view_info(actual_log)

(59026370, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59026370 entries, 42 to 105414946
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         uint32        
 1   event           object        
 2   timestamp       datetime64[ns]
 3   date_cd         datetime64[ns]
 4   application_id  uint32        
 5   total_applied   uint8         
dtypes: datetime64[ns](2), object(1), uint32(2), uint8(1)
memory usage: 2.3+ GB



,user_id,event,timestamp,date_cd,application_id,total_applied
42,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
43,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
44,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
45,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2
46,72878,EndLoanApply,2022-03-25 11:14:44,2022-03-25,871260,2


In [ ]:
view_info(no_log)

(46388577, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46388577 entries, 0 to 105414922
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   user_id         uint32        
 1   event           object        
 2   timestamp       datetime64[ns]
 3   date_cd         datetime64[ns]
 4   application_id  uint32        
 5   total_applied   uint8         
dtypes: datetime64[ns](2), object(1), uint32(2), uint8(1)
memory usage: 1.8+ GB



,user_id,event,timestamp,date_cd,application_id,total_applied
0,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
1,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
2,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
3,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0
4,576409,StartLoanApply,2022-03-25 11:12:09,2022-03-25,87171,0


In [ ]:
del app_log_merge
gc.collect()

752

## Get path

In [ ]:
def get_path(df:pd.DataFrame) -> pd.DataFrame:
    df.sort_values(by=['application_id', 'timestamp'], inplace=True)
    event_tuples = dict()
    for appid in tqdm(df['application_id'].unique()):
        subdf = df[df['application_id'] == appid]
        for dt in subdf['date_cd'].unique():
            subsubdf = subdf[subdf['date_cd'] == dt]
            events = tuple(subsubdf['event'])
            if events not in event_tuples:
                event_tuples[events] = 1
            else:
                event_tuples[events] += 1

    result = pd.DataFrame({'path': event_tuples.keys(), 'count': event_tuples.values()})

    return result

In [ ]:
app_path = get_path(actual_log)
app_path.to_csv("../Data/path_applied_user.csv", index=False)

In [ ]:
noapp_path = get_path(no_log)
noapp_path.to_csv("../Data/path_no_applied_user.csv", index=False)

In [ ]:
app_path.head(10)

In [ ]:
app_path.sort_values(by='count', ascending=False, inplace=True)
app_path.reset_index(drop=True, inplace=True)
for i in range(20):
    path, count = app_path.head(20).iloc[i]
    print(f"{path}\t{count}")

In [ ]:
noapp_path.head(10)

In [ ]:
noapp_path.sort_values(by='count', ascending=False, inplace=True)
noapp_path.reset_index(drop=True, inplace=True)
for i in range(20):
    path, count = noapp_path.head(20).iloc[i]
    print(f"{path}\t{count}")

## Merge loan path group and application id

In [ ]:
app_user = {user['application_id'].iloc[i]: user['user_id'].iloc[i] for i in range(user.shape[0])}
loan['user_id'] = loan['application_id'].map(app_user)
loan.dropna(subset=['user_id'], inplace=True)
loan['user_id'] = loan['user_id'].astype(np.uint32)

In [ ]:
loan

,application_id,loanapply_insert_time,is_applied,user_id
0,1748340,2022-06-07 13:05:41,NaN,430982
1,1748340,2022-06-07 13:05:41,NaN,430982
2,1748340,2022-06-07 13:05:41,NaN,430982
3,1748340,2022-06-07 13:05:41,NaN,430982
4,1748340,2022-06-07 13:05:41,NaN,430982
...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,NaN,478802
13527359,1428218,2022-06-03 12:01:49,NaN,478802
13527360,1428218,2022-06-03 12:01:48,NaN,478802
13527361,1428218,2022-06-03 12:01:48,NaN,478802


In [ ]:
path_group = pd.read_csv("../Data/path_group.csv", engine='c', low_memory=True)
view_info(path_group)

(1740729, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740729 entries, 0 to 1740728
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   application_id  float64
 1   path            object 
 2   date_cd         object 
 3   embeddings      object 
 4   user_id         int64  
 5   labels          int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 79.7+ MB



,application_id,path,date_cd,embeddings,user_id,labels
0,871260.0,"('Login', 'StartLoanApply', 'ViewLoanApplyIntr...",2022-03-25,[ 0.00453116 0.00014933 -0.00333886 0.007471...,72878,3
1,94017.0,"('OpenApp', 'Login', 'StartLoanApply', 'ViewLo...",2022-03-25,[ 0.0479816 0.11158732 -0.05752625 -0.063861...,618857,0
2,1482636.0,"('OpenApp', 'Login', 'StartLoanApply', 'ViewLo...",2022-03-25,[ 0.0479816 0.11158732 -0.05752625 -0.063861...,618857,0
3,1345169.0,"('OpenApp', 'Login', 'StartLoanApply', 'ViewLo...",2022-03-25,[ 0.00314156 0.09357454 -0.04824954 -0.061293...,496477,4
4,75852.0,"('OpenApp', 'Login', 'UseLoanManage', 'GetCred...",2022-03-25,[-0.00634183 0.07155684 -0.05163227 -0.045615...,217308,1


In [ ]:
null = path_group[path_group['application_id'].isna()]
no_null = path_group[~path_group['application_id'].isna()]

view_info(null)
view_info(no_null)

(1060733, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1060733 entries, 679996 to 1740728
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   application_id  0 non-null        float64
 1   path            1060733 non-null  object 
 2   date_cd         1060733 non-null  object 
 3   embeddings      1060733 non-null  object 
 4   user_id         1060733 non-null  int64  
 5   labels          1060733 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 56.6+ MB



,application_id,path,date_cd,embeddings,user_id,labels
679996,NaN,"('StartLoanApply', 'ViewLoanApplyIntro', 'Comp...",2022-03-24,[ 0.03146735 0.1536676 -0.08643018 -0.074777...,808079,0
679997,NaN,"('OpenApp', 'OpenApp', 'StartLoanApply', 'View...",2022-03-25,[ 0.01104167 0.04542439 -0.02805091 -0.040920...,808079,1
679998,NaN,"('OpenApp', 'StartLoanApply', 'ViewLoanApplyIn...",2022-03-26,[ 0.01934403 0.10600652 -0.05335192 -0.065669...,808079,4
679999,NaN,"('OpenApp', 'StartLoanApply', 'ViewLoanApplyIn...",2022-03-27,[ 0.01934403 0.10600652 -0.05335192 -0.065669...,808079,4
680000,NaN,"('OpenApp', 'GetCreditInfo', 'UseLoanManage')",2022-03-29,[ 0.02640858 0.05666207 -0.03640145 -0.016879...,808079,4


(679996, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679996 entries, 0 to 679995
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   application_id  679996 non-null  float64
 1   path            679996 non-null  object 
 2   date_cd         679996 non-null  object 
 3   embeddings      679996 non-null  object 
 4   user_id         679996 non-null  int64  
 5   labels          679996 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 36.3+ MB



,application_id,path,date_cd,embeddings,user_id,labels
0,871260.0,"('Login', 'StartLoanApply', 'ViewLoanApplyIntr...",2022-03-25,[ 0.00453116 0.00014933 -0.00333886 0.007471...,72878,3
1,94017.0,"('OpenApp', 'Login', 'StartLoanApply', 'ViewLo...",2022-03-25,[ 0.0479816 0.11158732 -0.05752625 -0.063861...,618857,0
2,1482636.0,"('OpenApp', 'Login', 'StartLoanApply', 'ViewLo...",2022-03-25,[ 0.0479816 0.11158732 -0.05752625 -0.063861...,618857,0
3,1345169.0,"('OpenApp', 'Login', 'StartLoanApply', 'ViewLo...",2022-03-25,[ 0.00314156 0.09357454 -0.04824954 -0.061293...,496477,4
4,75852.0,"('OpenApp', 'Login', 'UseLoanManage', 'GetCred...",2022-03-25,[-0.00634183 0.07155684 -0.05163227 -0.045615...,217308,1


In [ ]:
no_null['application_id'] = no_null['application_id'].astype('uint32')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
loan_ = pd.merge(left=loan, right=no_null[['application_id', 'labels']], how='left', on='application_id')
view_info(loan_)

(13545171, 5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13545171 entries, 0 to 13545170
Data columns (total 5 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   user_id                uint32        
 4   labels                 float64       
dtypes: datetime64[ns](1), float64(2), uint32(2)
memory usage: 516.7 MB



,application_id,loanapply_insert_time,is_applied,user_id,labels
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN


In [ ]:
null['labels_x'] = null['labels'].copy()
null.drop('labels', axis=1, inplace=True)
view_info(null)

(1060733, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1060733 entries, 679996 to 1740728
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   application_id  0 non-null        float64
 1   path            1060733 non-null  object 
 2   date_cd         1060733 non-null  object 
 3   embeddings      1060733 non-null  object 
 4   user_id         1060733 non-null  int64  
 5   labels_x        1060733 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 56.6+ MB



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,application_id,path,date_cd,embeddings,user_id,labels_x
679996,NaN,"('StartLoanApply', 'ViewLoanApplyIntro', 'Comp...",2022-03-24,[ 0.03146735 0.1536676 -0.08643018 -0.074777...,808079,0
679997,NaN,"('OpenApp', 'OpenApp', 'StartLoanApply', 'View...",2022-03-25,[ 0.01104167 0.04542439 -0.02805091 -0.040920...,808079,1
679998,NaN,"('OpenApp', 'StartLoanApply', 'ViewLoanApplyIn...",2022-03-26,[ 0.01934403 0.10600652 -0.05335192 -0.065669...,808079,4
679999,NaN,"('OpenApp', 'StartLoanApply', 'ViewLoanApplyIn...",2022-03-27,[ 0.01934403 0.10600652 -0.05335192 -0.065669...,808079,4
680000,NaN,"('OpenApp', 'GetCreditInfo', 'UseLoanManage')",2022-03-29,[ 0.02640858 0.05666207 -0.03640145 -0.016879...,808079,4


In [ ]:
loan_['date_cd'] = pd.to_datetime(loan_['loanapply_insert_time'].dt.date)
view_info(loan_)

(13545171, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13545171 entries, 0 to 13545170
Data columns (total 6 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   user_id                uint32        
 4   labels                 float64       
 5   date_cd                datetime64[ns]
dtypes: datetime64[ns](2), float64(2), uint32(2)
memory usage: 620.0 MB



,application_id,loanapply_insert_time,is_applied,user_id,labels,date_cd
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07


In [ ]:
null['date_cd'] = pd.to_datetime(null['date_cd'])
view_info(null)

(1060733, 6)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1060733 entries, 679996 to 1740728
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   application_id  0 non-null        float64       
 1   path            1060733 non-null  object        
 2   date_cd         1060733 non-null  datetime64[ns]
 3   embeddings      1060733 non-null  object        
 4   user_id         1060733 non-null  int64         
 5   labels_x        1060733 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 56.6+ MB



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,application_id,path,date_cd,embeddings,user_id,labels_x
679996,NaN,"('StartLoanApply', 'ViewLoanApplyIntro', 'Comp...",2022-03-24,[ 0.03146735 0.1536676 -0.08643018 -0.074777...,808079,0
679997,NaN,"('OpenApp', 'OpenApp', 'StartLoanApply', 'View...",2022-03-25,[ 0.01104167 0.04542439 -0.02805091 -0.040920...,808079,1
679998,NaN,"('OpenApp', 'StartLoanApply', 'ViewLoanApplyIn...",2022-03-26,[ 0.01934403 0.10600652 -0.05335192 -0.065669...,808079,4
679999,NaN,"('OpenApp', 'StartLoanApply', 'ViewLoanApplyIn...",2022-03-27,[ 0.01934403 0.10600652 -0.05335192 -0.065669...,808079,4
680000,NaN,"('OpenApp', 'GetCreditInfo', 'UseLoanManage')",2022-03-29,[ 0.02640858 0.05666207 -0.03640145 -0.016879...,808079,4


In [ ]:
loan_ = pd.merge(left=loan_, right=null[['user_id', 'date_cd', 'labels_x']], how='left', on=['user_id', 'date_cd'])
view_info(loan_)

(13545171, 7)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13545171 entries, 0 to 13545170
Data columns (total 7 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         uint32        
 1   loanapply_insert_time  datetime64[ns]
 2   is_applied             float64       
 3   user_id                uint32        
 4   labels                 float64       
 5   date_cd                datetime64[ns]
 6   labels_x               float64       
dtypes: datetime64[ns](2), float64(3), uint32(2)
memory usage: 723.4 MB



,application_id,loanapply_insert_time,is_applied,user_id,labels,date_cd,labels_x
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07,NaN
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07,NaN
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07,NaN
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07,NaN
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN,2022-06-07,NaN


In [ ]:
loan_.drop('date_cd', axis=1, inplace=True)

In [ ]:
(loan_['labels'] != loan_['labels_x']).sum()

13545171

In [ ]:
loan_.isnull().sum()

application_id                  0
loanapply_insert_time           0
is_applied                3257239
user_id                         0
labels                    4076082
labels_x                 12178195
dtype: int64

In [ ]:
loan_['labels_isnull'] = loan_['labels'].isnull()
loan_['labels_x_isnull'] = loan_['labels_x'].isnull()

loan_.head(10)

,application_id,loanapply_insert_time,is_applied,user_id,labels,labels_x,labels_isnull,labels_x_isnull
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True
5,1748340,2022-06-07 13:05:40,NaN,430982,NaN,NaN,True,True
6,1748340,2022-06-07 13:05:40,NaN,430982,NaN,NaN,True,True
7,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True
8,1748340,2022-06-07 13:05:46,NaN,430982,NaN,NaN,True,True
9,1748340,2022-06-07 13:05:51,NaN,430982,NaN,NaN,True,True


In [ ]:
loan_['label_null_agg'] = loan_['labels_isnull'].astype(np.uint8) + loan_['labels_x_isnull'].astype(np.uint8)
loan_.head()

,application_id,loanapply_insert_time,is_applied,user_id,labels,labels_x,labels_isnull,labels_x_isnull,label_null_agg
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2


In [ ]:
label_agg = np.zeros(shape=loan_.shape[0])
for i in tqdm(range(loan_.shape[0])):
    if loan_['label_null_agg'].iloc[i] == 2:
        label_agg[i] = loan_['labels'].iloc[i]
    elif loan_['label_null_agg'].iloc[i] == 1:
        if loan_['labels_isnull'].iloc[i]:
            label_agg[i] = loan_['labels_x'].iloc[i]
        else:
            label_agg[i] = loan_['labels'].iloc[i]
    else:
        label_agg[i] = loan_['labels'].iloc[i]

loan_['labels_agg'] = label_agg
loan_.head(10)

100%|██████████| 13545171/13545171 [38:28<00:00, 5868.71it/s]


,application_id,loanapply_insert_time,is_applied,user_id,labels,labels_x,labels_isnull,labels_x_isnull,label_null_agg,labels_agg
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2,NaN
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2,NaN
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2,NaN
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2,NaN
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2,NaN
5,1748340,2022-06-07 13:05:40,NaN,430982,NaN,NaN,True,True,2,NaN
6,1748340,2022-06-07 13:05:40,NaN,430982,NaN,NaN,True,True,2,NaN
7,1748340,2022-06-07 13:05:41,NaN,430982,NaN,NaN,True,True,2,NaN
8,1748340,2022-06-07 13:05:46,NaN,430982,NaN,NaN,True,True,2,NaN
9,1748340,2022-06-07 13:05:51,NaN,430982,NaN,NaN,True,True,2,NaN


In [ ]:
loan_.isnull().sum()

application_id                  0
loanapply_insert_time           0
is_applied                3257239
user_id                         0
labels                    4076082
labels_x                 12178195
labels_isnull                   0
labels_x_isnull                 0
label_null_agg                  0
labels_agg                2709106
dtype: int64

In [ ]:
loan_.drop(['labels', 'labels_x', 'labels_isnull', 'labels_x_isnull', 'label_null_agg'], axis=1, inplace=True)
loan_['labels'] = loan_['labels_agg'].copy()
loan_.drop('labels_agg', axis=1, inplace=True)

loan_.head(10)

,application_id,loanapply_insert_time,is_applied,user_id,labels
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN
5,1748340,2022-06-07 13:05:40,NaN,430982,NaN
6,1748340,2022-06-07 13:05:40,NaN,430982,NaN
7,1748340,2022-06-07 13:05:41,NaN,430982,NaN
8,1748340,2022-06-07 13:05:46,NaN,430982,NaN
9,1748340,2022-06-07 13:05:51,NaN,430982,NaN


## Deployment CSV

In [ ]:
loan_.to_csv('../Data/loan_.csv', index=False)

# Restart

In [ ]:
loan_ = pd.read_csv("../Data/loan_.csv", engine='c', low_memory=True)
loan_.head()

,application_id,loanapply_insert_time,is_applied,user_id,labels
0,1748340,2022-06-07 13:05:41,NaN,430982,NaN
1,1748340,2022-06-07 13:05:41,NaN,430982,NaN
2,1748340,2022-06-07 13:05:41,NaN,430982,NaN
3,1748340,2022-06-07 13:05:41,NaN,430982,NaN
4,1748340,2022-06-07 13:05:41,NaN,430982,NaN


In [ ]:
def user_separate(df:pd.DataFrame, key:str) -> list:
    app_subdfs = []
    for app in tqdm(df[key].unique()):
        subdf = df[df[key] == app]
        app_subdfs.append(subdf)
        
    return app_subdfs

In [ ]:
def fill_group(array:list) -> Union[int, pd.DataFrame]:
    null_users = 0
    concat_loan = pd.DataFrame()
    for item in tqdm(array):
        try:
            to_fill = item['labels'].value_counts().index[0].astype(np.uint8)
            item.fillna(to_fill, inplace=True)
            concat_loan = pd.concat([concat_loan, item], axis=0)
        except:
            null_users += 1
            concat_loan = pd.concat([concat_loan, item], axis=0)
    return concat_loan, null_users

In [ ]:
actual_users = user_separate(loan_, key='user_id')

threshold = len(actual_users) // 8
listA = actual_users[:threshold]
listB = actual_users[threshold:2*threshold]
listC = actual_users[2*threshold:3*threshold]
listD = actual_users[3*threshold:4*threshold]
listE = actual_users[4*threshold:5*threshold]
listF = actual_users[5*threshold:6*threshold]
listG = actual_users[6*threshold:7*threshold]
listH = actual_users[7*threshold:]

100%|██████████| 317467/317467 [1:14:39<00:00, 70.87it/s]


In [ ]:
resultA, nullA = fill_group(listA)
resultB, nullB = fill_group(listB)
resultC, nullC = fill_group(listC)
resultD, nullD = fill_group(listD)
resultE, nullE = fill_group(listE)
resultF, nullF = fill_group(listF)
resultG, nullG = fill_group(listG)
resultH, nullH = fill_group(listH)

100%|██████████| 39686/39686 [10:01<00:00, 66.01it/s]


In [ ]:
concat_loan = pd.concat([resultA, resultB, resultC, resultD, resultE, resultF, resultG, resultH], axis=0)
concat_loan.reset_index(drop=True, inplace=True)
view_info(concat_loan)

(13545171, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13545171 entries, 0 to 13545170
Data columns (total 5 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   application_id         int64  
 1   loanapply_insert_time  object 
 2   is_applied             float64
 3   user_id                int64  
 4   labels                 float64
dtypes: float64(2), int64(2), object(1)
memory usage: 516.7+ MB



,application_id,loanapply_insert_time,is_applied,user_id,labels
0,1748340,2022-06-07 13:05:41,0.0,430982,0.0
1,1748340,2022-06-07 13:05:41,0.0,430982,0.0
2,1748340,2022-06-07 13:05:41,0.0,430982,0.0
3,1748340,2022-06-07 13:05:41,0.0,430982,0.0
4,1748340,2022-06-07 13:05:41,0.0,430982,0.0


In [ ]:
null_users = nullA + nullB + nullC + nullD + nullE + nullF + nullH + nullG
null_users

31602

In [ ]:
concat_loan['labels'].value_counts()

0.0    4122604
4.0    3818921
1.0    2637573
3.0    1136156
5.0    1079598
2.0     153689
Name: labels, dtype: int64

In [ ]:
concat_loan[concat_loan['labels'].isna()].shape

(596630, 5)

## Deployment CSV

In [ ]:
concat_loan.to_csv("../Data/loan_path_segment.csv", index=False)